### Use pyhomogenize to check netCDF file(s) time axis; `time_control`

Now, we want to use pyhomogenize's `time_control` class. We open a test netCDF file. This will be done automatically by calling the class.

In [ ]:
import pyhomogenize as pyh

In [ ]:
time_control = pyh.time_control(pyh.test_netcdf[0])
time_control.ds

Let's have a look on the datasets's time axis

In [ ]:
time_control.time

We can check whether the time axis contains duplicated, missing or redundant time steps. A redundant time step is a time steps that does not math with the dataset's calendar and/or frequency.

In [ ]:
duplicates = time_control.get_duplicates()
redundants = time_control.get_redundants()
missings = time_control.get_missings()

In [ ]:
duplicates, redundants, missings

We see the time axis doesn't contain any incorrect time steps and no time steps are missing. Not really a auspicious example. We can combine the three above requests by using the function `check_timestamps`.

In [ ]:
timechecker1 = time_control.check_timestamps()
timechecker1

As we can see the functions returns a `time_control` object again but with three new attributes.

In [ ]:
timechecker1.duplicated_timesteps, timechecker1.missing_timesteps, timechecker1.redundant_timesteps

We want to test the time axis only for duplicated time steps.

timechecker2 = time_control.check_timestamps(selection='duplicates')
timechecker2.duplicated_timesteps

By setting the parameter correct to the boolean value `True` we can delete the duplicated and redundant time steps if exisitng. Of course, in our great example this is not the case.

In [ ]:
timechecker3 = time_control.check_timestamps(correct=True)
timechecker3.time

We can set the parameter `output` to select the dataset's output file name on disk. If so the parameter `correct` is automatically set to `True`.

In [ ]:
timechecker4 = time_control.check_timestamps(output="output.nc")

Now, we want to sleect a specific time range. We copy out `time_control` object to keep the original object. 

In [ ]:
from copy import copy

time_control1 = copy(time_control)
selected1 = time_control1.select_time_range(["2007-02-01", "2007-03-30"])
selected1

Here again, we get a `time_control` object. But now with a different time axis.

In [ ]:
selected1.time

Of course, we can write the result as netCDF file on disk. 

In [ ]:
time_control2 = copy(time_control)
selected2 = time_control2.select_time_range(
    ["2007-02-01", "2007-03-30"], output="output.nc"
)
selected2.ds

If we want to crop or limit the time axis to a user-specified start and end month values as shown in the above example `basics.date_range_to_frequency_limits` we can do this with netCDF files as well. The time axis should start with the start of an arbitrary season and end with the end of an arbitrary season.

In [ ]:
time_control3 = copy(time_control)
selected3 = time_control3.select_limited_time_range(
    smonth=[3, 6, 9, 12], emonth=[2, 5, 8, 11], output="output.nc"
)
selected3.time

Now, we want to check whether the time axis is within certain left and right bounds.

In [ ]:
time_control.within_time_range(["2007-02-01", "2007-03-30"])

In [ ]:
time_control.within_time_range(["2007-02-01", "2008-03-30"])

In [ ]:
time_control.within_time_range(["20070201", "20070330"], fmt="%Y%m%d")